# Web scraping of .cif files

Goal: to get .cif files of a material in each space group from the TQC.org database.

## Method 1: materials in TQC database

Strategy:

1. Search a space group in TQC.org, pick the first returned material and retrieve its ICSD identified (ICSD id.)
2. Search for this ICSD id. in the ICSD database and retrieve the .cif
3. Save the .cif 

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [ ]:
url = "https://topologicalquantumchemistry.org/#/?filter_sg_id=12"

driver = webdriver.Firefox()
driver.get(url)
time.sleep(5)
page_source = driver.page_source
driver.quit()

print("Page source retrieved successfully!")
print(page_source[:100])

#### Later in ICSD

In [ ]:
# URL of the page
url = "https://foo/bar.xhtml"

# Headers to mimic a browser (optional but recommended)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Data to send in the POST request (replace `input_name` with the actual form field name)
data = {
    "content_form:uiCodeCollection:input:input": "43456",  # The value to send
}

# Send a POST request with the data
response = requests.post(url, headers=headers, data=data)

# Parse the response
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    print("Response received!")
    print(soup.prettify())  # Print the prettified response (or extract what you need)
else:
    print(f"Request failed with status code: {response.status_code}")

## Method 2: scrap Materials Project

In [14]:
from mp_api.client import MPRester
import pyfplo.fedit as fedit
import pyfplo.fploio as fploio
import os
import shutil

In [15]:
API_KEY = '7hwV32FqdTRd4JKCJn2qI30qDzYvLukG'  # log in to MP and look for it there
mpr = MPRester(API_KEY)

Let's retrieve the smallest number of sites for materials in each space group (interrupted, takes quite long...)

In [52]:
list_nsites = []
for num_sg in range(1, 231):
    list_nsites_local = []
    materials = mpr.materials.summary.search(spacegroup_number=num_sg, num_magnetic_sites=0)
    for mat in materials:
        list_nsites_local.append(mat.nsites)
    list_nsites.append(min(list_nsites_local))
print(f'The largest nsites found: {min(list_nsites)}')

Retrieving SummaryDoc documents:   0%|          | 0/15014 [00:00<?, ?it/s]/home/mi2/envs/venv_irrep/lib/python3.10/site-packages/mp_api/client/core/client.py:841: UserWarning: Use the 'fields' argument to select only fields of interest to speed up data retrieval for large queries. Choose from: ['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'property_name', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure', 'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'es_source_calc_id', 'bandstructure', 'dos', 'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magn

In [16]:
for num_sg in range(1, 231):
    print(f'SG {num_sg:3d}')
    materials = mpr.materials.summary.search(spacegroup_number=num_sg,
                                             num_magnetic_sites=0,
                                             theoretical=True)
    if len(materials) == 0:
        print(f'SG {num_sg}: no material found.')
    try:
        id = materials[0].material_id
    except IndexError:
        print('No material found')
        continue
    cif = mpr.get_structure_by_material_id(id).to(fmt="cif")
    with open(f'{num_sg}.cif', 'w') as f:
        f.write(cif)

SG   1


Retrieving SummaryDoc documents:   0%|          | 0/14867 [00:00<?, ?it/s]

/home/mi2/envs/venv_irrep/lib/python3.10/site-packages/mp_api/client/core/client.py:841: UserWarning: Use the 'fields' argument to select only fields of interest to speed up data retrieval for large queries. Choose from: ['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'property_name', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure', 'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'es_source_calc_id', 'bandstructure', 'dos', 'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_nor

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

SG   2


...

Retrieving SummaryDoc documents:   0%|          | 0/92 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

SG 230


Retrieving SummaryDoc documents:   0%|          | 0/233 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Import CIF file and create a =.in with its crytal structure

In [2]:
def printsettings():
    fio=fploio.FPLOInput('=.in')
    par=fio.parser()
    d=par()
    print( 'spacegroup number : ',d('spacegroup.number').S)
    print( 'spacegroup setting: ',d('spacegroup.setting').S)
    print( 'lattice constants : ',d('lattice_constants').listS)
    print( 'axis angle        : ',d('axis_angles').listS)
    dw=d('wyckoff_positions')
    print( 'Wyckoff positions: ',dw.size())
    for i in range(dw.size()):
        taus=dw[i]('tau').listS
        print( '{0:>2s} {1:>20s} {2:>20s} {3:>20s}'.format(dw[i]('element').S,taus[0],taus[1],taus[2]))

In [27]:
tol_wp = 1e-4

list_failed = []

for num_sg in range(1, 231):

    print(num_sg)
    if num_sg == 168 or num_sg == 207: continue  # no material in MP for 168 :S
    
    try:

        # Create =.in if it doesn't exits, parse cif and write into =.in
        fio=fploio.FPLOInput('=.in')
        fio.structureFromCIFFile(f'{num_sg}.cif', wyckofftolerance=tol_wp, determinesymmetry=True)
        fio.writeFile("=.in")
            
        # Change other parameters in =.in
        fed=fedit.Fedit(recreate=False)
        fed.iteration(n=100)
        fed.relativistic('s')  # no SOC
        fed.vxc('5')
        fed.pipeFedit()
        
        # Print info about crystal
        printsettings()
        
        # Create sg_ directory and move =.in into it
        in_file = '=.in'
        dir = f'sg_{num_sg}'
        
        if not os.path.exists(dir):
            os.makedirs(dir)
        
        if os.path.exists(in_file):
            destination = os.path.join(dir, os.path.basename(in_file))
            shutil.move(in_file, destination)
        else:
            print('=.in file does not exist')
        

    except:
        list_failed.append(num_sg)

1
fedit pipe succesfull
spacegroup number :  1
spacegroup setting:  default
lattice constants :  ['6.002268', '10.39779975', '19.46632734']
axis angle        :  ['90.26591436', '90.04204249999999', '90.37922129']
Wyckoff positions:  74
Ag              0.00134             -0.00159              0.41118
Ag            -0.005649             0.006076             -0.25562
Ag             0.250703            -0.250682             0.000261
Ag             0.486035             0.499791             0.254972
Ag             0.493707            -0.494107            -0.255757
Ag            -0.247904            -0.249791             0.000741
Ag            -0.250872            -0.252652             0.334711
Ag            -0.249166            -0.249507            -0.333619
Ag                    0              0.49969            -0.000935
Ag                    0                  1/2                 -1/3
Ag             0.002472            -0.000141             0.255323
Ag             0.007501            -0.

In [26]:
list_failed

[]

Let's download of other materials in the SGs that failed. Change index until list_failed is empty

In [24]:
index = 2
for num_sg in list_failed:
    print(f'SG {num_sg:3d}')
    materials = mpr.materials.summary.search(spacegroup_number=num_sg, num_magnetic_sites=0)
    try:
        id = materials[index].material_id
    except IndexError:
        print('No material found')
        continue
    cif = mpr.get_structure_by_material_id(id).to(fmt="cif")
    with open(f'{num_sg}.cif', 'w') as f:
        f.write(cif)

SG 151


Retrieving SummaryDoc documents:   0%|          | 0/15 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Now, if we create the =.in for these groups, it also works.

### Analysis of the test

In [17]:
def error_percentage(file_log='log_failed_spacegroups.dat'):

    f = open(file_log, 'r')
    list_failed = []
    for line in f:
        list_failed.append(int(line))
    
    print(f'# Successfull: {230-len(list_failed)}')
    print(f'# Error: {len(list_failed)}')
    print(f'Success rate: {100-len(list_failed)/2.3} %')
    print(f'Failed SGs:')
    for sg in list_failed:
        print(sg)

In [18]:
error_percentage()

# Successfull: -44
# Error: 274
Success rate: -19.130434782608702 %
Failed SGs:
1
2
4
6
7
8
9
11
13
18
21
24
25
26
27
28
29
30
31
32
33
34
35
37
38
39
40
41
42
43
44
45
46
52
54
56
65
66
68
70
72
73
74
79
80
81
82
87
88
92
93
94
97
98
102
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
128
131
133
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
178
179
180
181
182
185
187
188
189
190
192
196
202
203
204
206
207
215
216
217
218
219
220
227
230
1
2
4
6
7
8
9
11
13
18
21
24
25
26
27
28
29
30
31
32
33
34
35
37
38
39
40
41
42
43
44
45
46
52
54
56
65
66
68
70
72
73
74
79
80
81
82
87
88
92
93
94
97
98
102
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
128
131
133
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
178
179
180
181
182
185
187
188
189
190
192
19